In [2]:
import mlflow
# Connect to Databricks workspace with submitted credentials or use stored credentials
mlflow.login()
# Set tracking URI to Databricks -  tell MLflow to send the data into Databricks Workspace
mlflow.set_tracking_uri("databricks")

2025/10/22 01:23:04 INFO mlflow.utils.credentials: Successfully connected to MLflow hosted tracking server! Host: https://dbc-7d1169bb-4536.cloud.databricks.com.


In [5]:
import os;
from datetime import datetime
from dotenv import load_dotenv

#Do not use .env file in shared databricks environment
#https://medium.com/@generative_ai/environment-variables-setting-in-databricks-dde16e3c3888 
load_dotenv()

experiment_name = f"/Users/{os.environ['USER_EMAIL']}/test-experiment"
try:
    mlflow.create_experiment(experiment_name)
except mlflow.exceptions.MlflowException:
    print(f"Experiment {experiment_name} already exists.")

mlflow.set_experiment(experiment_name)


with mlflow.start_run(run_name=f'run-{datetime.now()}') as run:
    mlflow.log_param("param1", 5)
    mlflow.log_metric("metric1", 0.85)

Experiment /Users/marijo.maracic@gmail.com/test-experiment already exists.
🏃 View run run-2025-10-22 01:26:20.756403 at: https://dbc-7d1169bb-4536.cloud.databricks.com/ml/experiments/3044475861801323/runs/2978f0bd71824bc3bd8cf4306874deeb
🧪 View experiment at: https://dbc-7d1169bb-4536.cloud.databricks.com/ml/experiments/3044475861801323
🏃 View run run-2025-10-22 01:26:20.756403 at: https://dbc-7d1169bb-4536.cloud.databricks.com/ml/experiments/3044475861801323/runs/2978f0bd71824bc3bd8cf4306874deeb
🧪 View experiment at: https://dbc-7d1169bb-4536.cloud.databricks.com/ml/experiments/3044475861801323


In [6]:
import pandas as pd
df = pd.read_csv('data/train.csv')
df.head()

,Date,store,product,number_sold
0,2010-01-01,0,0,801
1,2010-01-02,0,0,810
2,2010-01-03,0,0,818
3,2010-01-04,0,0,796
4,2010-01-05,0,0,808


In [8]:
# Count unique combinations of store and product
unique_combinations = df[['store','product']].drop_duplicates().shape[0]
print(f"Number of unique (store, product) combinations: {unique_combinations}")

# Count rows per (store, product) combination, sort descending
counts = df.groupby(['store', 'product']).size().reset_index(name='count')
counts_sorted = counts.sort_values('count', ascending=False)
display(counts_sorted)

# Show min and max counts
min_count = counts_sorted['count'].min()
max_count = counts_sorted['count'].max()
print(f"Min rows per (store, product): {min_count}")
print(f"Max rows per (store, product): {max_count}")

Number of unique (store, product) combinations: 70


,store,product,count
0,0,0,3287
1,0,1,3287
2,0,2,3287
3,0,3,3287
4,0,4,3287
...,...,...,...
65,6,5,3287
66,6,6,3287
67,6,7,3287
68,6,8,3287


Min rows per (store, product): 3287
Max rows per (store, product): 3287


In [10]:
# Show minimum and maximum date in the dataset
print('Min date:', df['Date'].min())
print('Max date:', df['Date'].max())

Min date: 2010-01-01
Max date: 2018-12-31


In [16]:
from statsmodels.tsa.arima.model import ARIMA
from tqdm import tqdm

results = {}
models = {}
groups = list(df.groupby(['store', 'product']))
for (store, product), group in tqdm(groups, desc='Training ARIMA models'):
    group_sorted = group.sort_values('Date')
    group_sorted = group_sorted.set_index('Date')
    try:
        model = ARIMA(group_sorted['number_sold'], order=(1,1,1))
        fit = model.fit()
        models[(store, product)] = fit
        forecast = fit.forecast(steps=1)[0]
        results[(store, product)] = forecast
    except Exception as e:
        models[(store, product)] = None
        results[(store, product)] = f'Error: {e}'

df_results = pd.DataFrame([
    {'store': k[0], 'product': k[1], 'forecast': v} for k, v in results.items()
])
display(df_results)

Training ARIMA models:   0%|          | 0/70 [00:00<?, ?it/s]/home/marijo/.cache/pypoetry/virtualenvs/databricks-timeseries-ddx-2ipW-py3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/marijo/.cache/pypoetry/virtualenvs/databricks-timeseries-ddx-2ipW-py3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/marijo/.cache/pypoetry/virtualenvs/databricks-timeseries-ddx-2ipW-py3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/tmp/ipykernel_34713/361797098.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future ver

,store,product,forecast
0,0,0,840.743832
1,0,1,705.695164
2,0,2,973.385224
3,0,3,886.574732
4,0,4,979.197684
...,...,...,...
65,6,5,898.873548
66,6,6,963.214415
67,6,7,832.240485
68,6,8,826.216882


In [17]:
import pandas as pd
test_df = pd.read_csv('data/test.csv')
test_df.head()

,Date,store,product,number_sold
0,2019-01-01,0,0,845
1,2019-01-02,0,0,851
2,2019-01-03,0,0,840
3,2019-01-04,0,0,842
4,2019-01-05,0,0,845


In [18]:
# Statistics for store, product combinations in test data
unique_combinations_test = test_df[['store','product']].drop_duplicates().shape[0]
print(f"Number of unique (store, product) combinations in test data: {unique_combinations_test}")

counts_test = test_df.groupby(['store', 'product']).size().reset_index(name='count')
counts_test_sorted = counts_test.sort_values('count', ascending=False)
display(counts_test_sorted)

min_count_test = counts_test_sorted['count'].min()
max_count_test = counts_test_sorted['count'].max()
print(f"Min points per (store, product) in test: {min_count_test}")
print(f"Max points per (store, product) in test: {max_count_test}")

Number of unique (store, product) combinations in test data: 70


,store,product,count
0,0,0,365
1,0,1,365
2,0,2,365
3,0,3,365
4,0,4,365
...,...,...,...
65,6,5,365
66,6,6,365
67,6,7,365
68,6,8,365


Min points per (store, product) in test: 365
Max points per (store, product) in test: 365


In [19]:
# Generate 365 predictions for each (store, product) using trained models and compare with test data
import numpy as np
predictions = {}
for key, model in models.items():
    if model is not None:
        try:
            forecast = model.forecast(steps=365)
            predictions[key] = forecast
        except Exception as e:
            predictions[key] = f'Error: {e}'
    else:
        predictions[key] = None

# Compare predictions with test data and calculate error metrics
comparison_results = []
for (store, product), forecast in predictions.items():
    test_points = test_df[(test_df['store'] == store) & (test_df['product'] == product)].sort_values('Date')
    if isinstance(forecast, (list, pd.Series)) and len(test_points) > 0:
        actual = test_points['number_sold'].values[:365]
        pred = forecast[:len(actual)]
        mse = np.mean((actual - pred) ** 2)
        mae = np.mean(np.abs(actual - pred))
        rmse = np.sqrt(mse)
        comparison_results.append({
            'store': store,
            'product': product,
            'actual': actual,
            'predicted': pred,
            'mse': mse,
            'mae': mae,
            'rmse': rmse
        })

# Sort by descending MSE
comparison_results_sorted = sorted(comparison_results, key=lambda x: x['mse'], reverse=True)

# Show best and worst MSE value
if comparison_results_sorted:
    best = comparison_results_sorted[-1]
    worst = comparison_results_sorted[0]
    print(f"Best MSE: {best['mse']:.2f} (Store: {best['store']}, Product: {best['product']})")
    print(f"Worst MSE: {worst['mse']:.2f} (Store: {worst['store']}, Product: {worst['product']})")

# Show comparison and error metrics for first few combinations (highest MSE)
for result in comparison_results_sorted[:5]:
    print(f"Store: {result['store']}, Product: {result['product']}")
    print("Actual:", result['actual'])
    print("Predicted:", result['predicted'])
    print(f"MSE: {result['mse']:.2f}, MAE: {result['mae']:.2f}, RMSE: {result['rmse']:.2f}")
    print()

Best MSE: 119.26 (Store: 3, Product: 5)
Worst MSE: 8352.34 (Store: 1, Product: 1)
Store: 1, Product: 1
Actual: [698 690 693 722 731 720 709 689 709 699 742 721 726 724 706 705 692 726
 741 732 708 700 710 707 724 745 745 727 709 713 703 726 725 732 721 705
 708 719 742 730 728 711 701 700 724 734 760 766 740 709 695 726 737 749
 738 749 727 707 731 756 742 754 761 722 709 750 761 754 767 755 716 729
 743 767 758 792 757 752 739 746 778 779 771 771 760 735 775 778 805 786
 769 764 763 788 792 800 780 778 762 780 777 791 795 802 783 764 774 772
 782 801 798 798 790 797 779 812 810 815 804 790 799 799 814 811 820 805
 798 792 830 849 839 832 814 797 812 798 820 851 840 823 801 816 834 839
 865 844 818 816 812 834 832 852 847 839 829 835 834 838 842 853 836 827
 828 831 841 856 853 838 829 831 808 859 857 859 839 842 847 848 851 861
 856 830 830 835 857 848 864 861 852 824 834 840 844 869 854 855 823 826
 850 847 868 873 842 832 809 840 854 855 863 831 841 797 816 846 861 842
 843 842 835 